In [ ]:
import pandas as pd
from helpers import read_csv, load_pickled_data, pickle_data
import os
import matplotlib.pyplot as plt 
import plotly
import chart_studio.plotly as py
import plotly.graph_objects as go
import os, sys
from helpers import bfs, get_class_pairs, choose_node_based_on_neighbors
import numpy as np
from Graph import Graph, Node
from visualizations import visualize_graph_from_list, visualize_graph_df
best_feature_cols = [ "Local_feature_1", "Local_feature_40", "Local_feature_42",	"Local_feature_46",	"Local_feature_51",	"Local_feature_52",	"Local_feature_54",	"Local_feature_58",	"Local_feature_60",	"Local_feature_64",	"Local_feature_66",	"Local_feature_76",	"Local_feature_78",	"Local_feature_88",	"total_BTC"]
df_cols = ["txId"] + best_feature_cols + ["class"]
TIMESTEP = 20
DATASET_BASE = "dataset"
PICKLED_TIMESTEPS = os.path.join("exports", "split_on_timestep.pkl")

txs_classes:pd.DataFrame = read_csv(os.path.join(DATASET_BASE, "elliptic_txs_classes.csv"), ret_Dataframe=True)
txs_edgelist:pd.DataFrame = read_csv(os.path.join(DATASET_BASE, "elliptic_txs_edgelist.csv"), ret_Dataframe=True)
txs_features:pd.DataFrame =  read_csv(os.path.join(DATASET_BASE, "elliptic_txs_features.csv"), ret_Dataframe=True)\

merge_feats_class = pd.merge(txs_features, txs_classes, on='txId')
merge_feats_class.shape

print(f"Total Nodes df shape: {txs_features.shape}")
print(f"total edges df shape: {txs_edgelist.shape}")
merge_feats_class = pd.merge(txs_features, txs_classes, on='txId')
merge_feats_class.shape
#timestep_graphs = load_pickled_data(PICKLED_TIMESTEPS)
timestep_dataframe = merge_feats_class.loc[merge_feats_class['Time step'] == TIMESTEP].reset_index(drop=True)[df_cols]
timestep_edges = txs_edgelist[txs_edgelist['txId1'].isin(timestep_dataframe['txId'])].reset_index(drop=True)


In [ ]:
timestep_list = timestep_dataframe.values.tolist()

edge_list = timestep_edges.values.tolist()

In [ ]:


graph_of_interest = Graph()
graph_of_interest.create_from_lists(timestep_list, edge_list)
#check node with max children. (ONLY Children)
max_children = 0
node_of_interest:Node = None


In [ ]:

node_of_interest:Node = choose_node_based_on_neighbors(graph_of_interest, 4, label=1, only_children=True)
if node_of_interest is not None:
    total_disjoint_graphs = []
    total_nodes = graph_of_interest.node_txId_set.copy()
    result = bfs(graph_of_interest, node_of_interest.txId,keep_direction=True)
    pairs = list(result[0])
    visited_nodes = list(result[1])
    #print(len(pairs), len(visited_nodes))
    nodes_n_classes = []
    for n in visited_nodes:
        nodes_n_classes.append([n, graph_of_interest.nodes[n].label])
    df1 = pd.DataFrame(nodes_n_classes, columns=['txId', 'class'])
    # print(df1)
    visualize_graph_df(df1, txs_edgelist, f"Plot for node: {node_of_interest.txId} with total neighbours: {node_of_interest.get_num_of_neighbors(only_children=False)}")
    #visualize_graph_from_list(cls_pairs, pairs, f"Graph in timestep {TIMESTEP} that includes the node {node_of_interest.txId}")
else:
    print("No such node")

In [ ]:
from helpers import find_best_fitted_dist_for_features
#best_distributions = find_best_fitted_dist_for_features(timestep_dataframe[best_feature_cols], 200)
